#### 1. Problem (case study)

##### 1.1. Data Description.

> You are working as an analyst for an auto insurance company. The company has collected some data about its customers including their demographics, education, employment, policy details, vehicle information on which insurance policy is, and claim amounts. You will help the senior management with some business questions that will help them to better understand their customers, improve their services, and improve profitability.

##### 1.2. Goal.

The business objectives are to:
- retain customers
- analyze relevant customer data
- develop focused customer retention programs

Our task in this project is:
> Based on the analysis, take targeted actions to increase profitable customer response, retention, and growth.

#### 2. Getting data - read the `.csv` file.

Import the libraries & modules needed throughout the project and change global settings:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from scipy import stats
import statsmodels.api as sm


pd.set_option('max_columns', None)

Read `.csv` file:

In [ ]:
df = pd.read_csv('files_for_lab/csv_files/marketing_customer_analysis.csv')

#### 3. Data Cleaning & Wrangling. Exploratory Data Analysis.

##### 3.1. Standardize headers names.

Review the data

In [ ]:
print(df.head())

We can remove the *Unnamed: 0* column as it is identical to the index column:

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

Review the rest of the columns:

In [ ]:
print(df.columns)

We can fix the typo, change the capitalization to lowercase, and replace all spaces with underscores:

In [ ]:
df.rename(columns = {'EmploymentStatus': 'Employment Status'}, inplace=True)
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")

##### 3.2. Review and address the `NaN` values.

Find and review the `NaN` values in each column as a % of the total number of rows. However, before doing so, we will remove duplicate rows (i.e. rows with the same *customer* value), as they could skew the % of `NaN` values. We'll also set the *customer* column as the index for further reference.

In [ ]:
df.drop_duplicates(subset='customer', inplace=True)
df.set_index('customer', drop=True, inplace=True)

nan_values = df.isna().sum()
nan_values_percentage = nan_values * 100 / df.shape[0]
print(nan_values_percentage)

Given the *vehicle_type* column has 50% missing values, we can drop it.

In [ ]:
df.drop('vehicle_type', axis=1, inplace=True)

The rest of the columns only have ~3% `NaN` values each (This used to be 5% before duplicate rows were removed). Therefore, we can check how much data we would lose if we were to remove the rest of the `NaN` values:

In [ ]:
nan_df = df[(df['state'].isna()) | (df['response'].isna()) | 
                    (df['months_since_last_claim'].isna()) | (df['number_of_open_complaints'].isna()) |
                    (df['vehicle_class'].isna()) | (df['vehicle_size'].isna())]

print(nan_df.shape[0] * 100 / df.shape[0])

As the rows with at least one `NaN` value represent ~9% of the total number of rows (this used to be 16% before dropping duplicates), we cannot remove all of them. Let's check how many rows with `NaN` values the categorical columns _state_ and _response_ have:

In [ ]:
nan_df1 = df[(df['state'].isna()) | (df['response'].isna())]
print(nan_df1.shape[0] * 100 / df.shape[0])

Given the percentage is identical to the one initially calculated for each of the columns, we can conclude that wherever there is a `NaN` value in the *state* column, there is also a `NaN` value in the *response* column. Now, we will check if it's feasible to replace the `NaN` values with the mode of the columns:

In [ ]:
states_count = df['state'].value_counts()
response_count = df['response'].value_counts()
print(states_count)
print(response_count)

We could choose California as the default state, however Oregon is relatively close in numbers (this used to be the case before droping duplicates too). Therefore, we'll drop the rows, which will automatically remove the `NaN` values in the _response_ column:

In [ ]:
df = df[(df['state'].isna() == False) | (df['response'].isna() == False)]

We will follow the same logic for the *vehicle_class* and *vehicle_size* categorical columns:

In [ ]:
nan_df2 = df[(df['vehicle_class'].isna()) | (df['vehicle_size'].isna())]
print(nan_df2.shape[0] * 100 / df.shape[0])

vehicle_class_count = df['vehicle_class'].value_counts()
vehicle_size_count = df['vehicle_size'].value_counts()
print(vehicle_class_count)
print(vehicle_size_count)

Firstly, we confirmed that for each `NaN` value in the *vehicle_class* column, there is a corresponding `NaN` value in the *vehicle_size* column. Secondly, in this case the modes of the columns are further apart from the second most encountered value, so we can replace the `NaN` values with the columns mode.

In [ ]:
vehicle_class_mode = df['vehicle_class'].mode()[0]
vehicle_size_mode = df['vehicle_size'].mode()[0]

df['vehicle_class'] = df['vehicle_class'].fillna(vehicle_class_mode)
df['vehicle_size'] = df['vehicle_size'].fillna(vehicle_size_mode)

For the other two columns, we can replace the `NaN` values with the median. Before doing so, we will briefly check that the values are either integers or floats, so we can apply the median:

In [ ]:
print(df.dtypes)

median_months = df['months_since_last_claim'].median()
median_open_complaints = df['number_of_open_complaints'].median()

df['months_since_last_claim'] = df['months_since_last_claim'].fillna(median_months)
df['number_of_open_complaints'] = df['number_of_open_complaints'].fillna(median_open_complaints)

##### 3.3. Categorical Features.

We can see the categorical features by looking at what data we have in each column:

In [ ]:
df.head()

We can see that the categorical features are: *customer*, *state*, *response*, *coverage*, *education*, *employment_status*, *gender*, *location_code*, *marital_status*, *policy_type*, *policy*, *renew_offer_type*, *sales_channel*, *vehicle_class*, *vehicle_size*.

We can split these categorical features into:
- **nominal features:** *customer*, *state*, *response*, *employment_status*, *gender*, *location_code*, *marital_status*, *policy_type*, *sales_channel*, *vehicle_class*
* **ordinal features:** *coverage*, *education*, *policy*, *renew_offer_type*, *vehicle_size*

##### 3.4. Numerical features

We can also see that the numerical features are: *customer_lifetime_value*, *effective_to_date* (which can be converted to # of years, months, days), *income*, *monthly_premium_auto*, *months_since_last_claim*, *months_since_policy_inception*, *number_of_open_complaints*, *number_of_policies*. The *total_claim_amount* is also a numerical column, but not a feature, as it is the value we want to predict.

We can split the numerical features into discrete and continuous numerical data by looking at the number of unique values in each numerical column. First, however, we will check that the data is stored correctly in the dataframe:

In [ ]:
df.info()

We only need to convert the *effective_to_date* data to datetime format:

In [ ]:
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], errors='raise')

Now we can have a look at the number of unique values in each of the numerical columns:

In [ ]:
for column in df:
    if df[column].dtype != np.object:
        print(column, ":", len(df[column].unique()))

We can now see which of our numerical data is:
- **discrete:** *effective_to_date*, *monthly_premium_auto*, *months_since_last_claim*, *months_since_policy_inception*, *number_of_open_complaints*, *number_of_policies*
* **continuous:** *customer_lifetime_value*, *income*, *total_claim_amount* 

** Noting that the data we counted as discrete may not be inherently discrete, but it does take only very few unique values in comparison to the rest of the dataset (which has 10279 values).

##### 3.5. Data Exploration.

First off, we can get a sense of the data (ranges, variance, mean) by using the *describe* method:

In [ ]:
df.describe()

However, it might be easier to visualize all of the columns to get a sense of the data distribution:
- bar plots for categorical data
- distribution plots for numerical data

In [ ]:
for column in df:
    if df[column].dtype == np.object:
        x = df[column].unique()
        y = df[column].value_counts()
        fig, ax = plt.subplots(figsize = (12, 9))
        plt.title(column)
        plt.bar(x, y)
        plt.show()    
    elif df[column].dtype == np.number:
        sns.displot(df[column])

We can notice that the education data is not clean, as we have one category (*College*), that can mean either *Bachelor's*, *Master's*, or *Doctoral* degree. In this case, we'll choose to reduce the number of possible options to *College* and *High School or Below* so we don't treat the *College* category as completely separate from the other 3:

In [ ]:
def replace_education(x):
    if x in ['Doctor', 'Bachelor', 'Master']:
        x = 'College'
    return x

df['education'] = df['education'].apply(replace_education)

We can also notice that:
- most of the customers have been to college, are employed, live in suburban areas, and have responded 'No' (however we're unsure if this means they have responded 'No' to an insurance renewal or a different question)
- there's an almost equal number of male and female customers
- the insurances seem to be mostly corporate and typically offer basic coverage

It would make sense that people living in suburban areas are more likely to use a car to commute and by extension, also purchase a car policy insurance. Similarly, it makes sense to have more corporate insurance policies if most people are employed. Now, are there any other interesting points to be uncovered from the data? For example:
- What type of insurances do people in different states go for? Or coverage?
- Are different vehicle sizes associated with different insurance coverages or are they equally distributed?
- Are the renew offer types dependent on the sales channel?

In [ ]:
sns.catplot(x='state', hue='policy_type', data=df, kind='count')
sns.catplot(x='state', hue='coverage', data=df, kind='count')
sns.catplot(x='vehicle_size', hue='coverage', data=df, kind='count')
sns.catplot(x='sales_channel', hue='renew_offer_type', data=df, kind='count')

We will also look at the Pearson and Spearman correlations in our dataset and remove any colinear data:

In [ ]:
df_num = df.select_dtypes(np.number)

corr = df_num.corr()
fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, annot=True, mask=mask)
plt.show()

corr = df_num.corr('spearman')
fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, annot=True, mask=mask)
plt.show()

There's no obvious strong correlation between the numerical features, so we will perform Chi-squared test for our categorical variables:

In [ ]:
df_cat = df.select_dtypes(np.object)

def chi_squared_test(df_cat):
    corr_chi = pd.DataFrame([], columns=df_cat.columns, index=df_cat.columns)
    for column_1 in df_cat:
        for column_2 in df_cat:
            st_resp = pd.crosstab(df_cat[column_1], df_cat[column_2])
            chi2, p, dof, array = chi2_contingency(st_resp)
            corr_chi.loc[column_1, column_2] = round(p, 3)

    for column in corr_chi:
        corr_chi[column] = pd.to_numeric(corr_chi[column], errors='raise')

    fig, ax = plt.subplots(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_chi, dtype=bool))
    sns.heatmap(corr_chi, annot=True, mask=mask)
    plt.show()

chi_squared_test(df_cat)

We will remove *policy_type* and *vehicle_size* as they are clearly dependednt on *policy* and *vehicle_class*, respectively and have less detailed information:

In [ ]:
df.drop(['policy_type', 'vehicle_size'], axis=1, inplace=True)
df_cat = df.select_dtypes(np.object)
chi_squared_test(df_cat)

We did not remove any more columns as that greatly affected the linear regression score when tested.

#### 4. Processing Data

##### 4.1. Spotting and addressing outliers.

We will look at the outliers in the continuous numerical data using boxplots:

In [ ]:
continuous = ['customer_lifetime_value', 'income', 'total_claim_amount']

for i in range(0, len(continuous)):
    plt.figure(i)
    sns.boxplot(x=df[continuous[i]], data=df)

It seems that the *customer_lifetime_value* and *total_claim_amount* have plenty of outliers, so we will clean these columns:

In [ ]:
def remove_outliers(columns, df, threshold=1.5):
    for column in columns:
        upper = np.percentile(df[column],75)
        lower = np.percentile(df[column],25)
        iqr = upper - lower
        upper_limit = upper + threshold * iqr
        lower_limit = lower - threshold * iqr
        df = df[(df[column]>lower_limit) & (df[column]<upper_limit)]
    return df

remove_outliers(continuous, df)

##### 4.2. Normalization. 

Before applying the linear regression, we want to make sure that our data is normalized so as to not skew the model. Additionally, we will apply the Box-Cox method to slightly improve the R2 score, given that typically the distribution of errors is approximated to a normal distribution:

In [ ]:
def split_data(df):
    y = df['total_claim_amount']
    y = y.apply(pd.to_numeric, errors='ignore')
    X = df.drop('total_claim_amount', axis=1)
    X_num = X.select_dtypes(include=np.number)
    X_cat = X.select_dtypes(include='object')
    return X, y, X_num, X_cat

def normalize_data(X_num):
    transformer = MinMaxScaler().fit(X_num) 
    x_minmax = transformer.transform(X_num)
    X_num_norm = pd.DataFrame(x_minmax, columns=X_num.columns)
    return X_num_norm

def Box_Cox(X_num_norm):
    for column in X_num_norm:
        if len(X_num_norm[column].unique()) > 500:
            X_num_norm[column] = np.where(X_num_norm[column]<=0, np.nan, X_num_norm[column])        
            X_num_norm[column] = X_num_norm[column].fillna(X_num_norm[column].mean())
            X_num_norm[column], _ = stats.boxcox(X_num_norm[column])
    return X_num_norm


df_dummy = df.copy()
X, y, X_num, X_cat = split_data(df_dummy)
X_num_norm = normalize_data(X_num)
X_num_norm = Box_Cox(X_num_norm)

##### 4.3. Encoding Categorical Data.

We will use One-Hot Encoding to encode the categorical data, as ordinal encoding yielded worse results in the linear regression model (as per lab 7]):

In [ ]:
def encode_data(X_cat):
    encoder = OneHotEncoder(handle_unknown='error', drop='first')
    encoder.fit(X_cat)
    encoded = encoder.transform(X_cat).toarray()
    cat_encoded = pd.DataFrame(encoded)
    cat_encoded.columns = encoder.get_feature_names_out()
    return cat_encoded

cat_encoded = encode_data(X_cat)

##### 4.4. Splitting into train set and test set.

In [ ]:
X = pd.concat([X_num_norm, cat_encoded], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### 5. Run Linear Regression Model

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions  = model.predict(X_test)

#### 6. Model Validation

In [ ]:
r2 = r2_score(y_test, predictions)
RMSE = mean_squared_error(y_test, predictions, squared=False)
MSE = mean_squared_error(y_test, predictions)
MAE = np.mean(abs(y_test.to_numpy() - predictions))

print("r2 = ", r2)
print("RMSE = ", RMSE)
print("MSE = ", MSE)
print("MAE = ", MAE)

#### 7. Reporting - present results.

The R2 score for the model means that ~77.95% of the variation in the *total_claim_amount* can be explained by variations in the rest of the variables. Whilst this is not a bad result, it could benefit from some improvement, either by:
- gathering more data 
* changing the model used to fit the data
- exploring other variables that might influence the dataset

By looking at the coefficients, we can see which variables have the most weight in the model:

In [ ]:
coefficients = pd.DataFrame(data=model.coef_, index=X.columns)
print(coefficients)

We can see that the *monthly_premium_auto* has the largest weight in our model and it's positively correlated with the *total_claim_amount*, suggesting that customers with a higher premium also claim more overall. Most of the other features have a less than 10x smaller weight in the model, suggesting they're not as indicative of the *total_claim_amount*, even when compounded (e.g. in the case of categorical columns, which were one-hot encoded). This might be explained by the fact that higher monthly premium autos also involve the possibility to claim more in the case of an accident. Therefore, the results do not point to an obvious course of action for the business, other than gathering more data or potentially using industry data. 